<a href="https://colab.research.google.com/github/johnprasanth93/NLP_Know_Your_Neighbours/blob/master/Know_Your_Neighbourhood_Task_1_AuthenticOrNot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating Twitter credentials JSON

In [ ]:
import json

In [ ]:
credentials = {
    'API_key' : 'REPLACE_IT_WITH_CREDENTIALS',
    'API_secret_key' :'REPLACE_IT_WITH_CREDENTIALS',
    'Access_token' : 'REPLACE_IT_WITH_CREDENTIALS-R4wED15V1ehBWhEtcOBPE5yJgticTse',
    'Access_token_secret' : 'REPLACE_IT_WITH_CREDENTIALS'
}

In [ ]:
with open('twitter_cred.json','w') as fp:
  json.dump(credentials,fp,indent=2)

## Loading Twitter Credentials

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving banglore_tweets.txt to banglore_tweets.txt
Saving indian_express_news.txt to indian_express_news.txt
User uploaded file "banglore_tweets.txt" with length 13143 bytes
User uploaded file "indian_express_news.txt" with length 4212 bytes


## Tweepy scrapper

In [ ]:
import tweepy

In [ ]:
auth = tweepy.OAuthHandler(credentials['API_key'], credentials['API_secret_key'])
auth.set_access_token(credentials['Access_token'], credentials['Access_token_secret'])

In [ ]:
auth

In [ ]:
api = tweepy.API(auth)

In [ ]:
api

In [ ]:
# API to collect 50 trending tweets at (Where on Earth ID - WOEID) location #Mumbai	2295411
trending_tweets = api.trends_place(2295411)
trending_tweets

[{'as_of': '2020-05-20T16:01:27Z',
  'created_at': '2020-05-20T15:53:47Z',
  'locations': [{'name': 'Mumbai', 'woeid': 2295411}],
  'trends': [{'name': 'Nepal',
    'promoted_content': None,
    'query': 'Nepal',
    'tweet_volume': 84845,
    'url': 'http://twitter.com/search?q=Nepal'},
   {'name': '#HappyBirthdayMohanlal',
    'promoted_content': None,
    'query': '%23HappyBirthdayMohanlal',
    'tweet_volume': 508670,
    'url': 'http://twitter.com/search?q=%23HappyBirthdayMohanlal'},
   {'name': '#योगी_की_हजार_ट्रेन',
    'promoted_content': None,
    'query': '%23%E0%A4%AF%E0%A5%8B%E0%A4%97%E0%A5%80_%E0%A4%95%E0%A5%80_%E0%A4%B9%E0%A4%9C%E0%A4%BE%E0%A4%B0_%E0%A4%9F%E0%A5%8D%E0%A4%B0%E0%A5%87%E0%A4%A8',
    'tweet_volume': 51564,
    'url': 'http://twitter.com/search?q=%23%E0%A4%AF%E0%A5%8B%E0%A4%97%E0%A5%80_%E0%A4%95%E0%A5%80_%E0%A4%B9%E0%A4%9C%E0%A4%BE%E0%A4%B0_%E0%A4%9F%E0%A5%8D%E0%A4%B0%E0%A5%87%E0%A4%A8'},
   {'name': '#AmphanSuperCyclone',
    'promoted_content': None,
    'q

In [ ]:
with open('trending_topics.json','w') as fp:
  json.dump(trending_tweets,fp,indent=4)

In [ ]:
# Trigger API to collect tweets about Banglore earthquake
keyword = '%23Bangalore'
tweets = api.search(q=keyword,count=100,lang='en')

In [ ]:
tweets[0]

Status(_api=<tweepy.api.API object at 0x7fb5fb948a58>, _json={'created_at': 'Wed May 20 16:22:43 +0000 2020', 'id': 1263143122286059521, 'id_str': '1263143122286059521', 'text': 'RT @beingbbt: Natural Disaster Management has said that it happens due to the vaccum of hot and cold air during the storm conditions. \nSadl…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'beingbbt', 'name': 'BB Baba', 'id': 2313199909, 'id_str': '2313199909', 'indices': [3, 12]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1158322435470422017, 'id_str': '1158322435470422017', 'name': 'Nikita', 'screen_name': 'heynikita_', 'location': 'Pune, India', 'de

In [ ]:
# Writing trending tweets into csv file
import csv
with open('trending_topics.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=' ')
    for tweet in tweets:
      writer.writerow([tweet])

In [ ]:
#Writing tweets for similarity check
with open('banglore_tweets.txt','a') as src_file:
  for tweet in tweets:
    src_file.write(tweet.text)

## Scraper to get contents from authentic websites

In [ ]:
!pip install boilerpy3

In [ ]:
from boilerpy3 import extractors
import re

In [ ]:
extractor = extractors.CanolaExtractor()

doc = extractor.get_doc_from_url('https://indianexpress.com/article/cities/bangalore/bengaluru-loud-sound-police-hul-investigation-6419246/')
page_title = doc.title
page_contents = doc.content

In [ ]:
with open('indian_express_news.txt','a') as fp:
  fp.write(page_title)
  fp.write('\n\n')
  fp.write(page_contents)

In [ ]:
print(page_title)

Bengaluru: Police investigating ‘loud sound’; earthquake ruled out | Cities News,The Indian Express


## Preprocessing the tweets and news

In [ ]:
with open('banglore_tweets.txt','r') as src:
  src_tweets = src.read()

with open('indian_express_news.txt','r') as news:
  express_news = news.read()

In [ ]:
# Remove Emojis from the tweets
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [ ]:
# Remove URLS mentioned in the tweets
def remove_url(inputString):
  return re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', inputString, flags=re.MULTILINE)

In [ ]:
src_tweets = remove_url(src_tweets)
src_tweets = deEmojify(src_tweets)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
# Download punkt tokeniser
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Tokenising the text
tweet_tokens = word_tokenize(src_tweets)
news_tokens = word_tokenize(express_news)

In [ ]:
# Lemmatization -> Converting verbs into its base form
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
wnl = WordNetLemmatizer()
lemmatized_tweet_tokens = []
lemmatized_news_tokens = []

for word in tweet_tokens:
    lemmatized_tweet_tokens.append(wnl.lemmatize(word, pos="v"))

for word in news_tokens:
    lemmatized_news_tokens.append(wnl.lemmatize(word, pos="v"))

tweet_tokens = lemmatized_tweet_tokens
news_tokens = lemmatized_news_tokens

In [ ]:
# Stop words removal
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(tweet_tokens)

['RT', '@', 'beingbbt', ':', 'Natural', 'Disaster', 'Management', 'have', 'say', 'that', 'it', 'happen', 'due', 'to', 'the', 'vaccum', 'of', 'hot', 'and', 'cold', 'air', 'during', 'the', 'storm', 'condition', '.', 'SadlRT', '@', 'vkjpandey', ':', 'That', 'sound', 'you', 'hear', '?', 'Do', "n't", 'worry', ',', 'it', 'be', 'just', '#', '2020', 'burp', 'after', 'eat', 'up', 'almost', 'half', 'of', 'the', 'year', '.', '#', 'earthquake', '#', 'bangalorPeople', 'surely', 'state', 'the', 'arrival', 'of', '#', 'Aliens', '&', 'amp', ';', 'an', '#', 'earthquake', 'take', 'place', 'in', '#', 'Bangalore', 'after', '#', 'bangaloreboom', '.', 'Mea', '@', 'imshivam2851', ':', 'Mtlb', '2020', 'toh', 'ekdum', 'lene', 'ke', 'liye', 'hath', 'dho', 'ke', 'peche', 'he', 'pad', 'gya', 'h', '...', '2020', 'mean', 'horrible', 'year', '#', 'Amphan', '#', 'Bangalore', '#', 'AmphanSuRT', '@', 'NewIndianXpress', ':', 'Was', 'it', 'a', 'plane', '?', 'An', '#', 'earthquake', '?', 'Was', 'it', '#', 'alien', '?', 'Th

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.',',','#','@',':','...','!','?','&','(',')','\'','..','--',';','-','“','``','”','‘','’','–','%'])
print(stop_words)

{'wasn', "you'll", '–', 'that', 'some', 'before', 'both', 'which', "you're", 'ain', 'can', 'i', 'between', 'herself', 'has', 'having', 'on', 'who', 'aren', 'in', 'just', "weren't", 'each', "couldn't", 'hadn', ':', 'ours', 't', 'him', 'its', 'during', "isn't", 'over', 'then', 'itself', 'theirs', "that'll", '(', 're', 'no', 'the', 'with', "needn't", '“', 'from', 'yours', 'didn', 'it', 'how', 'will', "hadn't", 'needn', 'shan', "hasn't", 'if', 'yourself', 'all', 'doing', 'y', 'you', 'himself', 'against', 'd', 'me', 'couldn', 'did', 'll', 'haven', 'have', 'not', 'below', 'wouldn', 'be', 'them', '”', 'such', "didn't", 'what', 'where', 'of', '-', 'above', 'than', 'is', 'only', "won't", 'again', "she's", 'ma', "mightn't", 'their', 'a', 'do', 'to', ',', 'as', 'about', 'her', "aren't", 'hers', 'been', 'at', "should've", "wouldn't", "haven't", "wasn't", 'but', 'being', 'your', 'off', 'whom', 'our', 'these', 'other', '...', 'should', 'because', 'why', 'own', "don't", 'shouldn', 'while', 'his', 's'

In [ ]:
filtered_tweet_tokens = []
for token in tweet_tokens:
  if token not in stop_words:
    filtered_tweet_tokens.append(token)

filtered_news_tokens = []
for token in news_tokens:
  if token not in stop_words:
    filtered_news_tokens.append(token)

In [ ]:
print(filtered_tweet_tokens)

['RT', 'beingbbt', 'Natural', 'Disaster', 'Management', 'say', 'happen', 'due', 'vaccum', 'hot', 'cold', 'air', 'storm', 'condition', 'SadlRT', 'vkjpandey', 'That', 'sound', 'hear', 'Do', "n't", 'worry', '2020', 'burp', 'eat', 'almost', 'half', 'year', 'earthquake', 'bangalorPeople', 'surely', 'state', 'arrival', 'Aliens', 'amp', 'earthquake', 'take', 'place', 'Bangalore', 'bangaloreboom', 'Mea', 'imshivam2851', 'Mtlb', '2020', 'toh', 'ekdum', 'lene', 'ke', 'liye', 'hath', 'dho', 'ke', 'peche', 'pad', 'gya', 'h', '2020', 'mean', 'horrible', 'year', 'Amphan', 'Bangalore', 'AmphanSuRT', 'NewIndianXpress', 'Was', 'plane', 'An', 'earthquake', 'Was', 'alien', 'The', 'boom', 'rattle', 'Bengaluru', 'pique', 'nation', 'Hear', 'AlsRT', 'mustaqeem31', 'That', 'nothing', 'Bangalore', 'sonic', 'boom', 'cause', 'air', 'raft', 'go', 'high', 'speed', 'Calm', 'guy', 'Bangalore', 'eaRT', 'Risky050', 'I', 'find', 'ways', 'survive', '2020', 'Bangalore', 'bangaloreboom', '2020', 'raginidwivedi24', 'RDWELF

In [ ]:
# Checking frequency distribution
from nltk import FreqDist

In [ ]:
FreqDist(filtered_tweet_tokens).most_common(20)

[('Bangalore', 87),
 ('sound', 28),
 ('hear', 27),
 ('earthquake', 27),
 ('Aliens', 26),
 ('BOOM', 22),
 ("n't", 18),
 ('boom', 18),
 ('see', 17),
 ('Bengaluru', 16),
 ('come', 16),
 ('Was', 14),
 ('happen', 13),
 ('2020', 13),
 ('sonicboom', 13),
 ('noise', 13),
 ('bangaloresound', 13),
 ('alien', 12),
 ('Loud', 12),
 ('The', 11)]

In [ ]:
FreqDist(filtered_news_tokens).most_common(20)

[('Bengaluru', 10),
 ('Police', 7),
 ('city', 7),
 ('say', 5),
 ("'s", 5),
 ('sound', 4),
 ('The', 4),
 ('Air', 4),
 ('Force', 4),
 ('fighter', 4),
 ('area', 4),
 ('across', 4),
 ('aircraft', 4),
 ('loud', 3),
 ('earthquake', 3),
 ('News', 3),
 ('Indian', 3),
 ('ask', 3),
 ('jet', 3),
 ('movement', 3)]

## Checking the similarity between tweets and news to check the authenticity

In [ ]:
# Checking for similarity using Jaccard index
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [ ]:
jaccard_score = jaccard_similarity(filtered_tweet_tokens,filtered_news_tokens)
print('Jaccard similarity score between the news and tweets : ',jaccard_score)

Jaccard similarity score between the news and tweets :  0.026604973973395025


In [ ]:
import spacy
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 81.6MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=91df1db9b5830abfa6c733aa1126566c70344e2deb5895724aa8ac00b123bee7
  Stored in directory: /tmp/pip-ephem-wheel-cache-qggsug3x/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import spacy
import en_vectors_web_lg
nlp = en_vectors_web_lg.load()

In [ ]:
# Spacy similarity
#spacy_obj = spacy.load('en')
tweets = nlp(tweets_str)
news = nlp(news_str)

similarity = round(tweets.similarity(news)*100,2)
#for token in tokens:
#    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

if similarity < 80 :
  print("The news is fake as per the analysis of {} tweets and {} News articles".format(len(src_tweets.split('\n')),len(express_news.split('\n'))))
else:
  print("The news is {}% Authentic as per the analysis of {} tweets and {} News articles".format(similarity,len(src_tweets.split('\n')),len(express_news.split('\n'))))

The news is 94.6% Authentic as per the analysis of 147 tweets and 35 News articles
